# Testing GPT + Analysis

### This is just a initial project and study. I'm not a professional in this area, so I'm just trying to learn and share my knowledge. I hope you enjoy it! :)

### This is just a initial project and study. I'm not a professional in this area, so I'm just trying to learn and share my knowledge. I hope you enjoy it! :)

In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())

True

This cell imports the libraries and the .env keys, which are needed for this project. The keys that we are using are from OpenAI and Pinecone.

The Data that we will use comes from Amazon, so I will upload it to GitHub. But you can download it from here: https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/ (Amazon Reviews Dataset) if you to get other samples. The ones that we are using here are the from the Amazon Fashion category.

In [2]:
# Extract data from files
data = []
with gzip.open('AMAZON_FASHION.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
        
metadata = []
with gzip.open('meta_AMAZON_FASHION.json.gz') as f:
    for l in f:
        metadata.append(json.loads(l.strip()))

Here is the explanation for the code above:

1. The first part of code is to load the Amazon Fashion dataset into a dictionary called data. 
2. The second block of code is to load the metadata for the Amazon Fashion dataset into a dictionary called metadata.

Both will be loaded in dataframes.

In [3]:
# Load the data to dataframes

df = pd.DataFrame.from_dict(data)
df = df[df['reviewText'].notna()]

df_meta=pd.DataFrame.from_dict(metadata)

Now, we are going to truncate the reviews so that we don't process too large or unuseful data. To do this, we will limit to 400 characters on the reviewText column.

In [4]:
# Truncate the reviewText

max_text_length=400
def truncate_review(text):
    return text[:max_text_length]

df['truncated']=df.apply(lambda row: truncate_review(row['reviewText']),axis=1)

In [5]:
# Look for productIds with enough reviews

df.groupby('asin').count().sort_values('overall')

,overall,verified,reviewTime,reviewerID,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,truncated
asin,,,,,,,,,,,,
0764443682,1,1,1,1,1,1,1,1,0,0,0,1
B019YM0O28,1,1,1,1,1,1,1,1,0,0,0,1
B019YME2N0,1,1,1,1,1,1,1,1,0,0,0,1
B019YMNS08,1,1,1,1,1,1,1,1,0,0,0,1
B00S7N99AY,1,1,1,1,1,1,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
B000PHANNM,2566,2566,2566,2566,2566,2566,2563,2566,85,2563,112,2566
B00RLSCLJM,3633,3633,3633,3633,3633,3633,3632,3633,225,3538,210,3633
B00I0VHS10,3884,3884,3884,3884,3884,3884,3880,3884,128,3872,107,3884


This cell is here just to find a product to use as an example. We will first use the second one from the bottom then the last one (from the bottom).

## Now, we will create the embeddings for the reviews and metadata.

In [6]:
# Work on only a slice of the dataframe

df = df.loc[df['asin'] == 'B000KPIHQ4'].copy()

Here, we will use HuggingFace to create the embeddings for the reviews. This is a demo on how we can do it without OpenAI. Later, we will use OpenAI to create the embeddings.

We will create a new column and insert into the dataframe. We will use the .apply() again.

In [7]:
# Import and apply embeddings from HuggingFace
# Warning! Be careful when/if applying embeddings from OpenAI like this - the full review dataframe is more than 800k rows.

from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

df['embeddings']=df.apply(lambda row: embeddings.embed_query(row['truncated']),axis=1)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
df

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,truncated,embeddings
11218,3.0,True,"09 26, 2007",A1CIM0XZ3UA926,B000KPIHQ4,M. Cane,"Good price, good product. Howver, it is generi...",Orthotics off the rack,1190764800,2,"{'Size Name:': ' Men's 5-5.5, Women's 7-7.5', ...",NaN,"Good price, good product. Howver, it is generi...","[-0.007861480116844177, -0.006780322175472975,..."
11219,5.0,True,"01 18, 2007",A1EVVPCWRW5YYZ,B000KPIHQ4,Deborah Morris,My husband rates these insoles a 5 for comfort...,Very comfortable,1169078400,3,"{'Size Name:': ' Men's 10-10.5, Women's 12', '...",NaN,My husband rates these insoles a 5 for comfort...,"[-0.07544194161891937, 0.025455087423324585, -..."
11220,5.0,True,"05 18, 2018",A2P3NZ9H4PANK0,B000KPIHQ4,Stephanie,I have worn the Powerstep Pinnacle shoe insole...,... Pinnacle shoe insoles for the past 5 years...,1526601600,NaN,"{'Size Name:': ' Men's 6-6.5, Women's 8-8.5', ...",NaN,I have worn the Powerstep Pinnacle shoe insole...,"[-0.06397589296102524, 0.012907429598271847, -..."
11221,1.0,True,"05 18, 2018",A2975GY186VV7A,B000KPIHQ4,jessica etim,Very uncomfortable feel like I wasted my money!,Uncomfortable,1526601600,NaN,"{'Size Name:': ' Men's 7-7.5, Women's 9-9.5', ...",NaN,Very uncomfortable feel like I wasted my money!,"[-0.009998543187975883, -0.05696876347064972, ..."
11222,5.0,True,"05 17, 2018",A3U8E58RIKWDAW,B000KPIHQ4,Nancy Mazzuca,work perfect,Five Stars,1526515200,NaN,"{'Size Name:': ' Men's 9-9.5, Women's 11-11.5'...",NaN,work perfect,"[-0.01534109003841877, -0.005922211799770594, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400874,5.0,True,"05 20, 2018",A7XUE85HN4TX0,B000KPIHQ4,Kentuckymom,My podiatrist recommended these Powerstep Inso...,Superb comfort and support,1526774400,NaN,"{'Size Name:': ' Men's 6-6.5, Women's 8-8.5', ...",NaN,My podiatrist recommended these Powerstep Inso...,"[-0.05021457374095917, -0.002803092123940587, ..."
400875,3.0,True,"05 20, 2018",A1KTBKOTBM436K,B000KPIHQ4,Amazon Customer,They worked well for the first few weeks and t...,Slid around in my shoes a bit,1526774400,NaN,"{'Size Name:': ' Men's 10-10.5, Women's 12', '...",NaN,They worked well for the first few weeks and t...,"[-0.04547719657421112, 0.01694941706955433, -0..."
400876,5.0,True,"05 20, 2018",A3S84SEN2Z6DZU,B000KPIHQ4,EJ,I think these are the best.,Five Stars,1526774400,NaN,"{'Size Name:': ' Men's 7-7.5, Women's 9-9.5', ...",NaN,I think these are the best.,"[-0.029356878250837326, 0.005295349285006523, ..."
400877,5.0,True,"05 19, 2018",A3JRCF1I92QLW0,B000KPIHQ4,Amazon Customer,"With plantar fasciitis, my feet hurt all the t...","With plantar fasciitis, my feet hurt all the t...",1526688000,NaN,"{'Size Name:': ' Men's 6-6.5, Women's 8-8.5', ...",NaN,"With plantar fasciitis, my feet hurt all the t...","[-0.03282293677330017, -0.0400298610329628, -0..."


In [ ]:
# Prepare training and test sets for training Random Forest Regressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(
    list(df.embeddings.values),
    df.overall,
    test_size = 0.2,
    random_state=1
)

In [ ]:
# Train and calculate mean absolute error

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=150)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mean_absolute_error(y_test, y_pred)

In [ ]:
# Import Pinecone client

import pinecone
from langchain.vectorstores import Pinecone

# Initialize Pinecone
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  
    environment=os.getenv('PINECONE_ENV')  
)

In [ ]:
# Create list with truncated review texts

texts=df['truncated'].tolist()

In [ ]:
# Send embedding vectors to Pinecone with Langchain

vstore = Pinecone.from_texts(texts, embeddings, index_name='cxanalytics')

In [ ]:
# Do a basic vector similarity search

query = "The quality is good"
result = vstore.similarity_search(query)
print(result)

In [ ]:
# Import RetrievalQA adn ChatOpenAPI and define review_chain in order to have GPT-4 access the review data

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4",temperature=0.0)
review_chain = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff", retriever=vstore.as_retriever())

In [ ]:
# Define the task for GPT-4 and run the chain

q="""
The reviews you see are for a product called 'Powerstep Pinnacle Orthotic Shoe Insoles'.
What is the overall impression of these reviews? Give most prevalent examples in bullets. 
What do you suggest we focus on improving?
"""

result=review_chain.run(q)
print(result)

In [ ]:
# Rename columns in dataframe and create metadatafield in order to do upserts with Pinecone's Python client directly

df=df.rename(columns={'embeddings':'values','reviewerID':'id'})
df['metadata']=df.apply(lambda row: dict(rating=row['overall']), axis=1)

In [ ]:
# Create two copies of data, one for the upsert and one for extracting reviewText from ids return from the filtered similarity search

data=df[['metadata','values','id']].to_dict(orient='records')
data_local=df[['metadata','values','reviewText','id']].to_dict(orient='records')

In [ ]:
# Create the Pinecone index

pinecone.create_index(name='filtered', metric='euclidean', dimension=768)
index = pinecone.Index('filtered')

In [ ]:
# Upload the data in batches of 50

from tqdm.auto import tqdm

for i in tqdm(range(0, len(data), 50)):
    j= i + 50
    if j > len(data):
        j = len(data)
    batch = data[i: j]
    index.upsert(vectors=batch)

In [ ]:
# Run a filtered similarity search

query=embeddings.embed_query("will buy again")
results = index.query(queries=[query], top_k=100, filter={'rating': {'$eq': 4.0}})
print(results)

In [ ]:
# Get the rating from id

get_rating_from_id = {
    x['id']: {
        'rating': x['metadata']['rating'],
        'review': x['reviewText'],
    } for x in data_local}

In [ ]:
# Python function that retrieves reviews matching query and specific rate

def review_and_rating(query,rating):
    query=embeddings.embed_query(query)
    results = index.query(queries=[query], top_k=100, filter={'rating': {'$eq': rating}})
    ids = [i['id'] for i in results['results'][0]['matches']]
    l=[]
    for i in ids:
        l.append(get_rating_from_id[i])
    return pd.DataFrame(l)

In [ ]:
# Repurchase list and winback list

repurchase_list=review_and_rating('will purchase again', 5.0)

winback=review_and_rating('disappointed', 1.0)
